In [ ]:
import numpy as np
from bilby.core.result import read_in_result
from bilby.hyper.model import Model
from gwpopulation.models.redshift import PowerLawRedshift
from tqdm import tqdm

from gwpop.mass_models import matter_matters_pairing
from gwpop.spin_models import (
    iid_spin_magnitude_gaussian,
    iid_spin_orientation_gaussian_isotropic,
)
from pipe.gwpopulation_pipe_pdb import draw_true_values

## 1. CONFIGURATION

In [ ]:
RESULT_FILE = "data/baseline5_widesigmachi2_mass_NotchFilterBinnedPairingMassDistribution_redshift_powerlaw_mag_iid_spin_magnitude_gaussian_tilt_iid_spin_orientation_result.hdf5"
N_SAMPLES = 1000  # int(1e6)
CHUNK_SIZE = int(1e3)
Z_MAX = 2.3


## 2. LOAD MAP PARAMETERS FROM BILBY RESULT FILE

In [ ]:
# Load "Broken Power Law + 2 Peaks model" result
result = read_in_result(RESULT_FILE)
post = result.posterior.copy()

In [ ]:
# --- MAP hyperparameters ---
# Extract the Maximum a Posteriori (MAP) parameters from a Bilby result.
# - Load the .hdf5 result file and copy the posterior samples.
# - Compute a score:
#     * If the prior is non-uniform, use log_likelihood + log_prior (true MAP).
#     * If the prior is uniform (constant, as in this case), log_prior adds nothing,
#       this means Maximum Likelihood (ML) and Maximum A Posteriori (MAP) coincide.
# - Select the sample that maximizes this score.

if "log_prior" in post and post["log_prior"].nunique() > 1:
    score = post.log_likelihood + post.log_prior
    max_likelihood_sample = post.iloc[np.argmax(score)]
else:
    max_likelihood_sample = post.iloc[np.argmax(post.log_likelihood)]

In [ ]:
# Set minimum and maximum allowed masses for the model (can be tuned)
max_likelihood_sample["absolute_mmin"] = 0.5
max_likelihood_sample["absolute_mmax"] = 350.0

In [ ]:
print("Max parameters:")
print(max_likelihood_sample["log_likelihood"])


## 3. CONSTRUCT POPULATION MODEL WITH MAP PARAMETERS

In [ ]:
# Compose the model: mass, spin orientation, spin magnitude, redshift
model = Model(
    [
        matter_matters_pairing,
        iid_spin_orientation_gaussian_isotropic,
        iid_spin_magnitude_gaussian,
        PowerLawRedshift(z_max=Z_MAX),
    ],
    cache=False,
)

model

In [ ]:
# Set the parameters from MAP sample
model.parameters.update(max_likelihood_sample)

## 4. DRAW SYNTHETIC POPULATION SAMPLES (IN CHUNKS)

In [ ]:
N_SAMPLES = int(100)
n_chunks = int(np.ceil(N_SAMPLES / CHUNK_SIZE))

all_events = []
for i in tqdm(range(n_chunks), desc="Simulating events"):
    current_chunk_size = min(CHUNK_SIZE, N_SAMPLES - i * CHUNK_SIZE)
    events = draw_true_values(model=model, vt_model=None, n_samples=current_chunk_size)
    # all_events.append(events)

In [ ]:
events